In [ ]:
import pandas as pd, numpy as np,xarray as xr
from pathlib import Path
import re, yaml, copy, json
import helper, events_methods
from sonpy import lib as sp
import subprocess
import plotly
from helper import RenderJSON
plotly.offline.init_notebook_mode()

In [ ]:
import itables
itables.init_notebook_mode(all_interactive=True )
itables.options.maxBytes = "1MB"
itables.options.lengthMenu = [25, 10, 50, 100, 200]
itables.options.buttons = ["copyHtml5", "csvHtml5", "excelHtml5"]
# itables.options.scrollY="200px"
# itables.options.scrollCollapse=True
# itables.options.paging=False
# itables.options.column_filters = "footer"
itables.options.layout={"topEnd": "pageLength", "top1": "searchBuilder"}

In [ ]:
params = yaml.safe_load(Path("params.yaml").open("r"))
spike2_path = Path(params["smrx_path"])
info_path = Path(params["config_path"])
res_path = Path(params["dest_path"])
RenderJSON(params)

In [ ]:
MyFile = sp.SonFile(str(spike2_path), True)
all_channels = pd.DataFrame([dict(
    name=MyFile.GetChannelTitle(i), 
    id=i, 
    chan_type=str(MyFile.ChannelType(i))[len("DataType."):],
    unit=MyFile.GetChannelUnits(i),
    size=MyFile.ChannelBytes(i),
    item_size=MyFile.ItemSize(i),
    scale=MyFile.GetChannelScale(i)/6553.6,
    offset=MyFile.GetChannelOffset(i),
    divide = MyFile.ChannelDivide(i),
)  for i in range(MyFile.MaxChannels()) if MyFile.ChannelType(i) != sp.DataType.Off])
all_channels



In [ ]:
info = yaml.safe_load(info_path.open("r"))
info

In [ ]:
channels_dict = {k+"_channel": grp["name"].to_list() for k, grp in all_channels.groupby("chan_type")}
processing = events_methods.EventProcessing.process_info(channels_dict, info["processing"] , dest_name="dest_channel")
processing_df = pd.DataFrame(list(processing.values()))
processing_df ["name"] = processing_df["method_params"].apply(lambda m: m["channel_name"])
processing_df  = processing_df.merge(all_channels, how="left", on="name")
processing_df

In [ ]:
# for dest_chan, item in channels_dict.items():
    